<a href="https://colab.research.google.com/github/MMathisLab/DeepLabCut/blob/master/examples/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590444465547-SHXODUII311HEE407IL6/ke17ZwdGBToddI8pDm48kE4VnnB9_j2k1VP236ADqAFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxQg9Vf0owGyf3dhfDKy8SxMujaKmp2B54Sb3VS1rO76Whq-cUhHVuKFlGUXsU9tJk/ezgif.com-video-to-gif.gif?format=1500w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub.

Mathis et al, in prep. <- please note, we are providing this toolbox as an early access release; more feeatures and details will be released with the forthcoming paper.




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#GUIs don't work on the cloud, so we supress them:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "5"
#os.environ["DLClight"]="True"
import deeplabcut
import tensorflow as tf

Loading DLC 2.2.2...


d:\Anaconda3\envs\DLC-GPU\lib\site-packages\deeplabcut\__init__.py:85: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  """


In [2]:
deeplabcut.__version__

'2.2.2'

## YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [3]:
# PLEASE EDIT THIS:
ProjectFolderName = 'Cricket16-Yuchen-2022-08-22'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path


#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/config.yaml'
path_config_file
#This creates a path variable that links to your google drive copy

'/sftpFolder/DeepLabCut/Cricket16-Yuchen-2022-08-22/config.yaml'

## Create a multi-animal training dataset:
### You must do this step inside of Colab:

- Reminder: you must connect EVERY bodypart in a skeleton before you run this step! See docs for crucial details on how to do this effciently: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#b-configure-the-project-

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589256735280-SCN7CROSJNJWCDS6EK5T/ke17ZwdGBToddI8pDm48kB08p9-rNkpPD7A3fw8YFjZZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PIno0kSvzOWihTW1zp8-1-7mzYxUQjsVr2n3nmNdVcso4/bodyparts-skeleton.png?format=1000w)
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589410182515-9SJO9MML6CNCXBAWQ6Z6/ke17ZwdGBToddI8pDm48kJ1oJoOIxBAgRD2ClXVCmKFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxBw7VlGKDQO2xTcc51Yv6DahHgScLwHgvMZoEtbzk_9vMJY_JknNFgVzVQ2g0FD_s/ezgif.com-video-to-gif+%2811%29.gif?format=750w)

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [5]:
#deeplabcut.cropimagesandlabels(path_config_file, size=(400, 400), userfeedback=False)
deeplabcut.add_new_videos(path_config_file, ['/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0495.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0496.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0497.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0498.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0499.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0500.mp4'], copy_videos=False)
#if you labeled on Windows, please set the windows2linux=True:

New video was added to the project! Use the function 'extract_frames' to select frames for labeling.


In [6]:
deeplabcut.extract_frames(path_config_file, mode='automatic', algo='kmeans', userfeedback=False, crop=True)

15it [00:00, 145.21it/s]

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 134.63  seconds.
Extracting and downsampling... 8070  frames from the video.


8070it [00:26, 301.59it/s]


Kmeans clustering ... (this might take a while)


d:\Anaconda3\envs\DLC-GPU\lib\site-packages\sklearn\cluster\_kmeans.py:888: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  f"MiniBatchKMeans is known to have a memory leak on "
34it [00:00, 329.60it/s]

...

Kmeans clustering ... (this might take a while)


32it [00:00, 308.51it/s]

Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 136.89  seconds.
Extracting and downsampling... 8205  frames from the video.


8205it [00:23, 354.12it/s]
d:\Anaconda3\envs\DLC-GPU\lib\site-packages\sklearn\cluster\_kmeans.py:888: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  f"MiniBatchKMeans is known to have a memory leak on "


Kmeans clustering ... (this might take a while)
Frames were successfully extracted, for the videos of interest.

You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [4]:
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


In [ ]:
deeplabcut.check_labels(path_config_file)

In [ ]:
deeplabcut.SkeletonBuilder(path_config_file)

In [4]:
deeplabcut.refine_labels(path_config_file)

Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file 

In [ ]:
deeplabcut.merge_datasets(path_config_file)

In [ ]:
deeplabcut.create_training_dataset(path_config_file, augmenter_type='imgaug', windows2linux=True)
#deeplabcut.create_multianimaltraining_dataset(path_config_file, windows2linux=False)

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. 
#Typically, you want to train to 50,000 iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

#which shuffle do you want to train?
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=1000, saveiters=1000, maxiters=300000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#%matplotlib notebook
#let's evaluate first:
%matplotlib inline
deeplabcut.evaluate_network(path_config_file,plotting=False)
#plot a few scoremaps:
#deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])


#and begin to cross-validate:
#deeplabcut.evaluate_multianimal_crossvalidate(
#            path_config_file,
#            Shuffles=[shuffle],
#            edgewisecondition=True,
#            leastbpts=1,
#            init_points=20,
#            n_iter=100,
#            target='rpck_train',
#        )

^ NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype='.MP4', save_as_csv=True)
deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype='.MP4')
deeplabcut.analyzeskeleton(path_config_file, videofile_path, videotype='.MP4', shuffle=1, save_as_csv=True)

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file, videofile_path)